# Finetuning FastPitch for a new speaker

In this tutorial, we will finetune a single speaker FastPitch (with alignment) model on 5 mins of a new speaker's data. We will finetune the model parameters only on the new speaker's text and speech pairs (though see the section at the end to learn more about mixing speaker data).

We will download the training data, then generate and run a training command to finetune Fastpitch on 5 mins of data, and synthesize the audio from the trained checkpoint.

A final section will describe approaches to improve audio quality past this notebook.

## License

> Copyright (c) 2021, NVIDIA CORPORATION & AFFILIATES.  All rights reserved.
>
> Licensed under the Apache License, Version 2.0 (the "License");
> you may not use this file except in compliance with the License.
> You may obtain a copy of the License at
>
>     http://www.apache.org/licenses/LICENSE-2.0
>
> Unless required by applicable law or agreed to in writing, software
> distributed under the License is distributed on an "AS IS" BASIS,
> WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
> See the License for the specific language governing permissions and
> limitations under the License.

In [1]:
import os
import json
import subprocess
from pathlib import Path

import numpy as np
import pandas as pd
from pydub import AudioSegment
from datasets import load_dataset, DatasetDict, Dataset

AUDIO_ROOT_PATH = Path("/home/antonio/entoa_prosodic/")

In [2]:
# Python wrapper to invoke the given bash script with the given input args
def run_script(script, args):
    args = ' \\'.join(args)
    cmd = f"python {script} \\{args}"

    print(cmd.replace(" \\", "\n"))
    print()
    !$cmd

## Downloading data & Create Manifest

Manifest format

```json
{
    "audio_filepath": "audio/dartagnan03part1_027_dumas_0047.wav", 
    "text": "yes monsieur", 
    "duration": 1.04, 
    "text_no_preprocessing": "Yes, monsieur.", 
    "text_normalized": "Yes, monsieur."
}
```

In [3]:
entoa_prosodic_ds = load_dataset("nilc-nlp/NURC-SP_ENTOA_TTS", name="prosodic", trust_remote_code=True)
entoa_prosodic_ds

DatasetDict({
    validation: Dataset({
        features: ['path', 'name', 'speaker', 'start_time', 'end_time', 'normalized_text', 'text', 'duration', 'type', 'year', 'gender', 'age_range', 'total_duration', 'quality', 'theme', 'audio'],
        num_rows: 473
    })
    train: Dataset({
        features: ['path', 'name', 'speaker', 'start_time', 'end_time', 'normalized_text', 'text', 'duration', 'type', 'year', 'gender', 'age_range', 'total_duration', 'quality', 'theme', 'audio'],
        num_rows: 7525
    })
})

In [4]:
# Get unique speakers and their counts
speakers, counts = np.unique(entoa_prosodic_ds['train']['speaker'], return_counts=True)

# Create a DataFrame with speakers and their counts
speaker_train_df = pd.DataFrame({
    'Speaker': speakers,
    'Count': counts
})

# Sort by count in descending order
speaker_train_df = speaker_train_df.sort_values('Count', ascending=False).reset_index(drop=True)

# Display the DataFrame
print(f"Total speakers: {len(speakers)}")
print(f"Total samples: {sum(counts)}")
speaker_train_df.head(5)  # Show top 10 speakers

Total speakers: 38
Total samples: 7525


,Speaker,Count
0,SP_D2_396_TB-L1,524
1,SP_D2_343_TB-L1,500
2,SP_DID_018_TB-L1,396
3,SP_EF_153_TB-L1,369
4,SP_D2_333_TB-L1,357


In [5]:
# Get unique speakers and their counts
speakers, counts = np.unique(entoa_prosodic_ds['validation']['speaker'], return_counts=True)

# Create a DataFrame with speakers and their counts
speaker_val_df = pd.DataFrame({
    'Speaker': speakers,
    'Count': counts
})

# Sort by count in descending order
speaker_val_df = speaker_val_df.sort_values('Count', ascending=False).reset_index(drop=True)

# Display the DataFrame
print(f"Total speakers: {len(speakers)}")
print(f"Total samples: {sum(counts)}")
speaker_val_df.head(5)  # Show top 10 speakers

Total speakers: 36
Total samples: 473


,Speaker,Count
0,SP_EF_153_TB-L1,38
1,SP_EF_388_TB-L1,33
2,SP_D2_343_TB-L1,29
3,SP_EF_124_TB-L1,28
4,SP_EF_405_TB-L1,27


`SP_D2_333_TB-DOC1` e `SP_DID_251_TB-DOC1` não estão no dataset de `validation`

In [7]:
print(set(speaker_train_df["Speaker"].to_list()).difference(speaker_val_df["Speaker"].to_list()),\
set(speaker_val_df["Speaker"].to_list()).difference(speaker_train_df["Speaker"].to_list())
)

common_speakers = set(speaker_train_df["Speaker"].to_list()).intersection(speaker_val_df["Speaker"].to_list())
len(common_speakers)
common_speakers

{'SP_D2_333_TB-DOC1', 'SP_DID_251_TB-DOC1'} set()


{'SP_D2_255_TB-DOC1',
 'SP_D2_255_TB-DOC2',
 'SP_D2_255_TB-L1',
 'SP_D2_255_TB-L2',
 'SP_D2_333_TB-L1',
 'SP_D2_333_TB-L2',
 'SP_D2_343_TB-DOC1',
 'SP_D2_343_TB-L1',
 'SP_D2_343_TB-L2',
 'SP_D2_360_TB-DOC1',
 'SP_D2_360_TB-L1',
 'SP_D2_360_TB-L2',
 'SP_D2_396_TB-DOC1',
 'SP_D2_396_TB-L1',
 'SP_D2_396_TB-L2',
 'SP_DID_018_TB-DOC1',
 'SP_DID_018_TB-L1',
 'SP_DID_137_TB-DOC1',
 'SP_DID_137_TB-L1',
 'SP_DID_161_TB-DOC1',
 'SP_DID_161_TB-L1',
 'SP_DID_208_TB-DOC1',
 'SP_DID_208_TB-L1',
 'SP_DID_235_TB-DOC1',
 'SP_DID_235_TB-L1',
 'SP_DID_242_TB-DOC1',
 'SP_DID_242_TB-L1',
 'SP_DID_250_TB-DOC1',
 'SP_DID_250_TB-L1',
 'SP_DID_251_TB-L1',
 'SP_EF_124_TB-L1',
 'SP_EF_153_TB-L1',
 'SP_EF_156_TB-L1',
 'SP_EF_377_TB-L1',
 'SP_EF_388_TB-L1',
 'SP_EF_405_TB-L1'}

In [8]:
def filter_dataset_by_speaker(dataset:Dataset, speaker_id:str) -> Dataset:
    """
    Filter a dataset to include only samples from a specific speaker.
    
    Args:
        dataset: A HuggingFace dataset object
        speaker_id: The ID of the speaker to filter for
        
    Returns:
        A filtered dataset containing only the specified speaker's data
    """
    # Create a filter function that checks if the speaker matches the given ID
    def speaker_filter(example):
        return example['speaker'] == speaker_id
    
    # Apply the filter to the dataset
    filtered_dataset = dataset.filter(speaker_filter)
    
    print(f"Original dataset size: {len(dataset)} samples")
    print(f"Filtered dataset size for speaker {speaker_id}: {len(filtered_dataset)} samples")
    
    return filtered_dataset

# Example usage
filtered_ds = filter_dataset_by_speaker(entoa_prosodic_ds['train'], "SP_D2_396_TB-L1")

Original dataset size: 7525 samples
Filtered dataset size for speaker SP_D2_396_TB-L1: 524 samples


In [9]:
def convert_wav_sample_rate(input_file:Path, output_file:Path, sample_rate:int):
    """
    Convert a WAV file to a different sample rate using pydub.
    
    Args:
        input_file (str): Path to the input WAV file
        output_file (str): Path to save the converted WAV file
        sample_rate (int): Target sample rate in Hz (e.g., 44100, 48000)
    
    Returns:
        bool: True if conversion was successful, False otherwise
    """
    assert input_file.exists()
    
    try:
        # Load the audio file
        audio:AudioSegment = AudioSegment.from_wav(input_file)

        if audio.frame_rate == sample_rate:
            return True
        
        # Set the new sample rate
        audio = audio.set_frame_rate(sample_rate)
        
        # Export with the new sample rate
        audio.export(output_file, format="wav")
        
        return True
    
    except Exception as e:
        print(f"Error during conversion: {e}")
        return False

In [5]:
audio_path = Path("/home/antonio/SP_D2_062_seg_3870.60_3871.36.wav")
audio = AudioSegment.from_wav(audio_path)
audio.frame_rate
# convert_wav_sample_rate(audio_path, audio_path, 22050)

22050

In [ ]:
def create_manifest(dataset:DatasetDict, split:str) -> tuple[Path, dict]:
    manifest_path = AUDIO_ROOT_PATH/f'{split}_manifest.json'
    speakers = dict()
    sid = 0

    if manifest_path.exists():
        os.remove(manifest_path)

    with open(AUDIO_ROOT_PATH/f'{split}_manifest.json', 'w+') as f:
        for sample in dataset:
            audio_path:Path = AUDIO_ROOT_PATH / sample['path']

            assert audio_path.exists()
            assert convert_wav_sample_rate(audio_path, audio_path, 22050)

            speakers[sample['speaker']] = sid

            json.dump({
                "audio_filepath": str(audio_path),
                "duration": float(sample['duration']),
                "text": sample['normalized_text'],
                "speaker": sid
            }, f)

            sid+=1
            f.write('\n')

    return manifest_path, speakers

```bash
sudo chmod -R 777 .
```

In [22]:
for speaker in common_speakers:
    print(f"\33[42m Processing speaker: {speaker} \33[0m")
    filtered_train_ds = filter_dataset_by_speaker(entoa_prosodic_ds['train'], speaker)
    manifest_train_path, sp1 = create_manifest(filtered_train_ds, 'train')

    filtered_val_ds = filter_dataset_by_speaker(entoa_prosodic_ds['validation'], speaker)
    manifest_validation_path, sp2 = create_manifest(filtered_val_ds, 'validation')
    break

 Processing speaker: SP_DID_161_TB-L1 
Original dataset size: 7525 samples
Filtered dataset size for speaker SP_DID_161_TB-L1: 193 samples


Original dataset size: 473 samples
Filtered dataset size for speaker SP_DID_161_TB-L1: 13 samples


In [112]:
len(sp1|sp2)

41

# Speaker Mapping

In [116]:
speaker_map_script = Path("/home/antonio/NeMo/scripts/dataset_processing/tts/") / "create_speaker_map.py"

train_manifest_filepath = manifest_train_path
dev_manifest_filepath = manifest_validation_path
speaker_filepath = AUDIO_ROOT_PATH / "speakers.json"

args = [
    f"--manifest_path={train_manifest_filepath}",
    f"--manifest_path={dev_manifest_filepath}",
    f"--speaker_map_path={speaker_filepath}"
]

run_script(speaker_map_script, args)

python /home/antonio/NeMo/scripts/dataset_processing/tts/create_speaker_map.py
--manifest_path=/home/antonio/entoa_prosodic/train_manifest.json
--manifest_path=/home/antonio/entoa_prosodic/validation_manifest.json
--speaker_map_path=/home/antonio/entoa_prosodic/speakers.json



In [ ]:
home_path = !(echo $HOME)
home_path = home_path[0]
print(home_path)

/home/antonio


# Training

In [12]:
import os
import json
from pathlib import Path

import torch
import IPython.display as ipd
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt

from nemo.collections.tts.models import FastPitchModel
FastPitchModel.from_pretrained("tts_en_fastpitch")

nemo_files = [p for p in Path(f"{home_path}/.cache/torch/NeMo/").glob("**/tts_en_fastpitch_align.nemo")]
nemo_files
# print(f"Copying {nemo_files[0]} to ./")
# Path("./tts_en_fastpitch_align.nemo").write_bytes(n qemo_files[0].read_bytes())

[NeMo I 2025-04-16 21:39:33 cloud:58] Found existing object /home/antonio/.cache/torch/NeMo/NeMo_2.1.0/tts_en_fastpitch_align/b7d086a07b5126c12d5077d9a641a38c/tts_en_fastpitch_align.nemo.
[NeMo I 2025-04-16 21:39:33 cloud:64] Re-using file from: /home/antonio/.cache/torch/NeMo/NeMo_2.1.0/tts_en_fastpitch_align/b7d086a07b5126c12d5077d9a641a38c/tts_en_fastpitch_align.nemo
[NeMo I 2025-04-16 21:39:33 common:826] Instantiating model from pre-trained checkpoint


 NeMo-text-processing :: INFO     :: Creating ClassifyFst grammars.
[NeMo W 2025-04-16 21:40:01 en_us_arpabet:66] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2025-04-16 21:40:01 modelPT:176] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /ws/LJSpeech/nvidia_ljspeech_train_clean_ngc.json
      sample_rate: 22050
      sup_data_path: /raid/LJSpeech/supplementary
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
 

[NeMo I 2025-04-16 21:40:01 features:305] PADDING: 1
[NeMo I 2025-04-16 21:40:02 save_restore_connector:275] Model FastPitchModel was successfully restored from /home/antonio/.cache/torch/NeMo/NeMo_2.1.0/tts_en_fastpitch_align/b7d086a07b5126c12d5077d9a641a38c/tts_en_fastpitch_align.nemo.


NameError: name 'home_path' is not defined

To finetune the FastPitch model on the above created filelists, we use the `examples/tts/fastpitch_finetune.py` script to train the models with the `fastpitch_align_v1.05.yaml` configuration.

Let's grab those files.

In [ ]:
# !wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/examples/tts/fastpitch_finetune.py

# !mkdir -p conf \
# && cd conf \
# && wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/examples/tts/conf/fastpitch_align_v1.05.yaml \
# && cd ..

We also need some additional files (see `FastPitch_MixerTTS_Training.ipynb` tutorial for more details) for training. Let's download these, too.

In [ ]:
# additional files
!mkdir -p tts_dataset_files && cd tts_dataset_files \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/scripts/tts_dataset_files/cmudict-0.7b_nv22.10 \
&& wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/scripts/tts_dataset_files/heteronyms-052722 \
&& cd ..

## Finetuning FastPitch

We can now train our model with the following command:

**NOTE: This will take about 50 minutes on colab's K80 GPUs.**

In [20]:
NEMO_DIR = Path('/home/antonio/NeMo/')
DATA_DIR = Path('/home/antonio/data/')

FASTPITCH_PATH = Path('/home/antonio/FastPitch/FastPitch.nemo')
FINETUNING_SCRIPT_PATH = NEMO_DIR / 'examples' / 'tts' / 'fastpitch_finetune.py'
CONFIG_TTS_PATH = NEMO_DIR / 'examples' / 'tts' / 'conf' / 'fastpitch_align_v1.05.yaml'

TRAIN_DATASET_PATH = DATA_DIR / '9017_manifest_train_dur_5_mins_local.json'
VALIDATION_DATASET_PATH = DATA_DIR / '9017_manifest_dev_ns_all_local.json'

SUP_DATA_PATH = DATA_DIR / 'fastpitch_sup_data/'
EXP_MANAGER_DIR = DATA_DIR / 'ljspeech_to_9017_no_mixing_5_mins'

CMU_DICT_PATH = NEMO_DIR / 'scripts' / 'tts_dataset_files' / 'ipa_cmudict-0.7b_nv23.01.txt'
HETERONYMS_PATH = NEMO_DIR / 'scripts' / 'tts_dataset_files' / 'heteronyms-052722'


assert all([
    NEMO_DIR.exists(),
    DATA_DIR.exists(),
    FASTPITCH_PATH.exists(),
    FINETUNING_SCRIPT_PATH.exists(),
    CONFIG_TTS_PATH.exists(),
    TRAIN_DATASET_PATH.exists(),
    VALIDATION_DATASET_PATH.exists(),
    CMU_DICT_PATH.exists(),
    HETERONYMS_PATH.exists(),
])

In [14]:
# If an error occurs, log the entire stacktrace.
os.environ["HYDRA_FULL_ERROR"] = "1"
os.environ["OC_CAUSE"] = "1"

```python
# TODO(oktai15): remove +model.text_tokenizer.add_blank_at=true when we update FastPitch checkpoint
!(python $FINETUNING_SCRIPT_PATH --config-name=fastpitch_align_v1.05.yaml \
  train_dataset=$manifest_train_path \
  validation_datasets=$manifest_validation_path \
  sup_data_path=$SUP_DATA_PATH \
  phoneme_dict_path=$CMU_DICT_PATH \
  heteronyms_path=$HETERONYMS_PATH \
  exp_manager.exp_dir=$EXP_MANAGER_DIR \
  +init_from_nemo_model=$FASTPITCH_PATH \
  +trainer.max_steps=1000 ~trainer.max_epochs \
  trainer.check_val_every_n_epoch=25 \
  model.train_ds.dataloader_params.batch_size=4 model.validation_ds.dataloader_params.batch_size=24 \
  model.n_speakers=41 model.pitch_mean=152.3 model.pitch_std=64.0 \
  model.pitch_fmin=30 model.pitch_fmax=512 model.optim.lr=2e-4 \
  ~model.optim.sched model.optim.name=adam trainer.devices=1 trainer.strategy=auto \
  +model.text_tokenizer.add_blank_at=true \
)
```

In [15]:
for speaker in common_speakers:
    print(f"\33[42m Processing speaker: {speaker} \33[0m")
    filtered_train_ds = filter_dataset_by_speaker(entoa_prosodic_ds['train'], speaker)
    manifest_train_path, sp1 = create_manifest(filtered_train_ds, 'train')

    filtered_val_ds = filter_dataset_by_speaker(entoa_prosodic_ds['validation'], speaker)
    manifest_validation_path, sp2 = create_manifest(filtered_val_ds, 'validation')
   
    # Build the command arguments
    cmd_args = [
        "python", str(FINETUNING_SCRIPT_PATH),
        "--config-name=fastpitch_align_v1.05.yaml",
        f"train_dataset={manifest_train_path}",
        f"validation_datasets={manifest_validation_path}",
        f"sup_data_path={SUP_DATA_PATH}",
        f"phoneme_dict_path={CMU_DICT_PATH}",
        f"heteronyms_path={HETERONYMS_PATH}",
        f"exp_manager.exp_dir={EXP_MANAGER_DIR}",
        f"+init_from_nemo_model={FASTPITCH_PATH}",
        "+trainer.max_steps=1000", "~trainer.max_epochs",
        "trainer.check_val_every_n_epoch=25",
        "model.train_ds.dataloader_params.batch_size=4", "model.validation_ds.dataloader_params.batch_size=24",
        "model.n_speakers=41", "model.pitch_mean=152.3", "model.pitch_std=64.0",
        "model.pitch_fmin=30", "model.pitch_fmax=512", "model.optim.lr=2e-4",
        "~model.optim.sched", "model.optim.name=adam", "trainer.devices=1", "trainer.strategy=auto",
        "+model.text_tokenizer.add_blank_at=true",
    ]

    # Print the command for reference
    print("Executing command:")
    print(" \\\n".join(cmd_args))
    print()

    # Run the command and capture output
    try:
        result = subprocess.run(
            cmd_args,
            check=True,
            text=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        print("Command executed successfully")
        print("Output:")
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Command failed with return code {e.returncode}")
        print("Error output:")
        print(e.stderr)
    
    break


 Processing speaker: SP_DID_161_TB-L1 
Original dataset size: 7525 samples
Filtered dataset size for speaker SP_DID_161_TB-L1: 193 samples
Original dataset size: 473 samples
Filtered dataset size for speaker SP_DID_161_TB-L1: 13 samples
Executing command:
python \
/home/antonio/NeMo/examples/tts/fastpitch_finetune.py \
--config-name=fastpitch_align_v1.05.yaml \
train_dataset=/home/antonio/entoa_prosodic/train_manifest.json \
validation_datasets=/home/antonio/entoa_prosodic/validation_manifest.json \
sup_data_path=/home/antonio/data/fastpitch_sup_data \
phoneme_dict_path=/home/antonio/NeMo/scripts/tts_dataset_files/cmudict-0.7b_nv22.10 \
heteronyms_path=/home/antonio/NeMo/scripts/tts_dataset_files/heteronyms-052722 \
exp_manager.exp_dir=/home/antonio/data/ljspeech_to_9017_no_mixing_5_mins \
+init_from_nemo_model=/home/antonio/FastPitch/FastPitch.nemo \
+trainer.max_steps=1000 \
~trainer.max_epochs \
trainer.check_val_every_n_epoch=25 \
model.train_ds.dataloader_params.batch_size=4 \
mod

In [25]:
for speaker in common_speakers:
    print(f"\33[42m Processing speaker: {speaker} \33[0m")
    filtered_train_ds = filter_dataset_by_speaker(entoa_prosodic_ds['train'], speaker)
    manifest_train_path, sp1 = create_manifest(filtered_train_ds, 'train')

    filtered_val_ds = filter_dataset_by_speaker(entoa_prosodic_ds['validation'], speaker)
    manifest_validation_path, sp2 = create_manifest(filtered_val_ds, 'validation')
   
    !(python $FINETUNING_SCRIPT_PATH --config-name=fastpitch_align_v1.05.yaml \
        train_dataset=$manifest_train_path \
        validation_datasets=$manifest_validation_path \
        sup_data_path=$SUP_DATA_PATH \
        phoneme_dict_path=$CMU_DICT_PATH \
        heteronyms_path=$HETERONYMS_PATH \
        exp_manager.exp_dir=$EXP_MANAGER_DIR \
        +init_from_nemo_model=$FASTPITCH_PATH \
        +trainer.max_steps=1000 ~trainer.max_epochs \
        trainer.check_val_every_n_epoch=25 \
        model.train_ds.dataloader_params.batch_size=24 model.validation_ds.dataloader_params.batch_size=24 \
        model.n_speakers=1 model.pitch_mean=152.3 model.pitch_std=64.0 \
        model.pitch_fmin=30 model.pitch_fmax=512 model.optim.lr=2e-4 \
        ~model.optim.sched model.optim.name=adam trainer.devices=1 trainer.strategy=auto \
        +model.text_tokenizer.add_blank_at=true \
    )
    
    break


 Processing speaker: SP_DID_161_TB-L1 
Original dataset size: 7525 samples
Filtered dataset size for speaker SP_DID_161_TB-L1: 193 samples


Original dataset size: 473 samples
Filtered dataset size for speaker SP_DID_161_TB-L1: 13 samples
[NeMo W 2025-04-16 23:23:56 nemo_logging:361] /opt/conda/envs/nemo/lib/python3.10/site-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_job(
    
[NeMo W 2025-04-16 23:23:56 nemo_logging:361] /opt/conda/envs/nemo/lib/python3.10/site-packages/lightning/fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
    
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
[NeMo I 2025-04-16 23:23:56 exp_manager:450] ExpManager schema
[NeMo I 2025-04-16 23:23:56 exp_manager:451] {'explicit_l

In [ ]:
# TODO(oktai15): remove +model.text_tokenizer.add_blank_at=true when we update FastPitch checkpoint
!(python $FINETUNING_SCRIPT_PATH --config-name=fastpitch_align_v1.05.yaml \
  train_dataset=$manifest_train_path \
  validation_datasets=$manifest_validation_path \
  sup_data_path=$SUP_DATA_PATH \
  phoneme_dict_path=$CMU_DICT_PATH \
  heteronyms_path=$HETERONYMS_PATH \
  exp_manager.exp_dir=$EXP_MANAGER_DIR \
  +init_from_nemo_model=$FASTPITCH_PATH \
  +trainer.max_steps=1000 ~trainer.max_epochs \
  trainer.check_val_every_n_epoch=25 \
  model.train_ds.dataloader_params.batch_size=4 model.validation_ds.dataloader_params.batch_size=4 \
  model.n_speakers=1 model.pitch_mean=152.3 model.pitch_std=64.0 \
  model.pitch_fmin=30 model.pitch_fmax=512 model.optim.lr=2e-4 \
  ~model.optim.sched model.optim.name=adam trainer.devices=1 trainer.strategy=auto \
  +model.text_tokenizer.add_blank_at=true \
)


[NeMo W 2025-04-16 23:06:05 nemo_logging:361] /opt/conda/envs/nemo/lib/python3.10/site-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_job(
    
[NeMo W 2025-04-16 23:06:05 nemo_logging:361] /opt/conda/envs/nemo/lib/python3.10/site-packages/lightning/fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
    
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
[NeMo I 2025-04-16 23:06:05 exp_manager:450] ExpManager schema
[NeMo I 2025-04-16 23:06:05 exp_manager:451] {'explicit_log_dir': None, 'exp_dir': None, 'name': None, 'version': None, 'use_datetime_version': True, 'resu

In [ ]:
# TODO(oktai15): remove +model.text_tokenizer.add_blank_at=true when we update FastPitch checkpoint
!(python fastpitch_finetune.py --config-name=fastpitch_align_v1.05.yaml \
  train_dataset=./9017_manifest_train_dur_5_mins_local.json \
  validation_datasets=./9017_manifest_dev_ns_all_local.json \
  sup_data_path=./fastpitch_sup_data \
  phoneme_dict_path=tts_dataset_files/cmudict-0.7b_nv22.10 \
  heteronyms_path=tts_dataset_files/heteronyms-052722 \
  exp_manager.exp_dir=./ljspeech_to_9017_no_mixing_5_mins \
  +init_from_nemo_model=./tts_en_fastpitch_align.nemo \
  +trainer.max_steps=1000 ~trainer.max_epochs \
  trainer.check_val_every_n_epoch=25 \
  model.train_ds.dataloader_params.batch_size=24 model.validation_ds.dataloader_params.batch_size=24 \
  model.n_speakers=1 model.pitch_mean=152.3 model.pitch_std=64.0 \
  model.pitch_fmin=30 model.pitch_fmax=512 model.optim.lr=2e-4 \
  ~model.optim.sched model.optim.name=adam trainer.devices=1 trainer.strategy=auto \
  +model.text_tokenizer.add_blank_at=true \
)

Let's take a closer look at the training command:

* `--config-name=fastpitch_align_v1.05.yaml`
  * We first tell the script what config file to use.

* `train_dataset=./9017_manifest_train_dur_5_mins_local.json 
  validation_datasets=./9017_manifest_dev_ns_all_local.json 
  sup_data_path=./fastpitch_sup_data`
  * We tell the script what manifest files to train and eval on, as well as where supplementary data is located (or will be calculated and saved during training if not provided).
  
* `phoneme_dict_path=tts_dataset_files/cmudict-0.7b_nv22.10 
heteronyms_path=tts_dataset_files/heteronyms-052722
`
  * We tell the script where `phoneme_dict_path` and `heteronyms-052722` are located. These are the additional files we downloaded earlier, and are used in preprocessing the data.
  
* `exp_manager.exp_dir=./ljspeech_to_9017_no_mixing_5_mins`
  * Where we want to save our log files, tensorboard file, checkpoints, and more.

* `+init_from_nemo_model=./tts_en_fastpitch_align.nemo`
  * We tell the script what checkpoint to finetune from.

* `+trainer.max_steps=1000 ~trainer.max_epochs trainer.check_val_every_n_epoch=25`
  * For this experiment, we tell the script to train for 1000 training steps/iterations rather than specifying a number of epochs to run. Since the config file specifies `max_epochs` instead, we need to remove that using `~trainer.max_epochs`.

* `model.train_ds.dataloader_params.batch_size=24 model.validation_ds.dataloader_params.batch_size=24`
  * Set batch sizes for the training and validation data loaders.

* `model.n_speakers=1`
  * The number of speakers in the data. There is only 1 for now, but we will revisit this parameter later in the notebook.

* `model.pitch_mean=152.3 model.pitch_std=64.0 model.pitch_fmin=30 model.pitch_fmax=512`
  * For the new speaker, we need to define new pitch hyperparameters for better audio quality.
  * These parameters work for speaker 9017 from the Hi-Fi TTS dataset.
  * If you are using a custom dataset, running the script `python <NeMo_base>/scripts/dataset_processing/tts/extract_sup_data.py manifest_filepath=<your_manifest_path>` will precalculate supplementary data and print these pitch stats.
  * fmin and fmax are hyperparameters to librosa's pyin function. We recommend tweaking these only if the speaker is in a noisy environment, such that background noise isn't predicted to be speech.

* `model.optim.lr=2e-4 ~model.optim.sched model.optim.name=adam`
  * For fine-tuning, we lower the learning rate.
  * We use a fixed learning rate of 2e-4.
  * We switch from the lamb optimizer to the adam optimizer.

* `trainer.devices=1 trainer.strategy=auto`
  * For this notebook, we default to 1 gpu which means that we do not need ddp.
  * If you have the compute resources, feel free to scale this up to the number of free gpus you have available.
  * Please remove the `trainer.strategy=auto` section if you intend on multi-gpu training.

## Synthesize Samples from Finetuned Checkpoints

Once we have finetuned our FastPitch model, we can synthesize the audio samples for given text using the following inference steps. We use a HiFi-GAN vocoder trained on LJSpeech.

We define some helper functions as well.

In [ ]:
from nemo.collections.tts.models import HifiGanModel
from nemo.collections.tts.models import FastPitchModel

vocoder = HifiGanModel.from_pretrained("tts_en_hifigan")
vocoder = vocoder.eval().cuda()

In [ ]:
def infer(spec_gen_model, vocoder_model, str_input, speaker=None):
    """
    Synthesizes spectrogram and audio from a text string given a spectrogram synthesis and vocoder model.
    
    Args:
        spec_gen_model: Spectrogram generator model (FastPitch in our case)
        vocoder_model: Vocoder model (HiFiGAN in our case)
        str_input: Text input for the synthesis
        speaker: Speaker ID
    
    Returns:
        spectrogram and waveform of the synthesized audio.
    """
    with torch.no_grad():
        parsed = spec_gen_model.parse(str_input)
        if speaker is not None:
            speaker = torch.tensor([speaker]).long().to(device=spec_gen_model.device)
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed, speaker=speaker)
        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)
        
    if spectrogram is not None:
        if isinstance(spectrogram, torch.Tensor):
            spectrogram = spectrogram.to('cpu').numpy()
        if len(spectrogram.shape) == 3:
            spectrogram = spectrogram[0]
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return spectrogram, audio

def get_best_ckpt_from_last_run(
        base_dir, 
        new_speaker_id, 
        duration_mins, 
        mixing_enabled, 
        original_speaker_id, 
        model_name="FastPitch"
    ):    
    mixing = "no_mixing" if not mixing_enabled else "mixing"
    
    d = f"{original_speaker_id}_to_{new_speaker_id}_{mixing}_{duration_mins}_mins"
    
    exp_dirs = list([i for i in (Path(base_dir) / d / model_name).iterdir() if i.is_dir()])
    last_exp_dir = sorted(exp_dirs)[-1]
    
    last_checkpoint_dir = last_exp_dir / "checkpoints"
    
    last_ckpt = list(last_checkpoint_dir.glob('*-last.ckpt'))

    if len(last_ckpt) == 0:
        raise ValueError(f"There is no last checkpoint in {last_checkpoint_dir}.")
    
    return str(last_ckpt[0])

Specify the speaker ID, duration of the dataset in minutes, and speaker mixing variables to find the relevant checkpoint (for example, we've saved our model in `ljspeech_to_9017_no_mixing_5_mins/`) and compare the synthesized audio with validation samples of the new speaker.

The mixing variable is False for now, but we include some code to handle multiple speakers for reference.

In [ ]:
new_speaker_id = 9017
duration_mins = 5
mixing = False
original_speaker_id = "ljspeech"

last_ckpt = get_best_ckpt_from_last_run("./", new_speaker_id, duration_mins, mixing, original_speaker_id)
print(last_ckpt)

spec_model = FastPitchModel.load_from_checkpoint(last_ckpt)
spec_model.eval().cuda()

# Only need to set speaker_id if there is more than one speaker
speaker_id = None
if mixing:
    speaker_id = 1

num_val = 2  # Number of validation samples
val_records = []
with open(f"{new_speaker_id}_manifest_dev_ns_all_local.json", "r") as f:
    for i, line in enumerate(f):
        val_records.append(json.loads(line))
        if len(val_records) >= num_val:
            break
            
for val_record in val_records:
    print("Real validation audio")
    ipd.display(ipd.Audio(val_record['audio_filepath'], rate=22050))
    print(f"SYNTHESIZED FOR -- Speaker: {new_speaker_id} | Dataset size: {duration_mins} mins | Mixing:{mixing} | Text: {val_record['text']}")
    spec, audio = infer(spec_model, vocoder, val_record['text'], speaker=speaker_id)
    ipd.display(ipd.Audio(audio, rate=22050))
    %matplotlib inline
    imshow(spec, origin="lower", aspect="auto")
    plt.show()

## Improving Speech Quality

We see that from fine-tuning FastPitch, we were able to generate audio in a male voice but the audio quality is not as good as we expect. We recommend two steps to improve audio quality:

* Finetuning HiFi-GAN
* Adding more data

**Note that both of these steps are outside the scope of the notebook due to the limited compute available on Colab, but the code is included below for you to use outside of this notebook.**

### Finetuning HiFi-GAN
From the synthesized samples, there might be audible audio crackling. To fix this, we need to finetune HiFi-GAN on the new speaker's data. HiFi-GAN shows improvement using **synthesized mel spectrograms**, so the first step is to generate mel spectrograms with our finetuned FastPitch model to use as input.

The code below uses our finetuned model to generate synthesized mels for the training set we have been using. You will also need to do the same for the validation set (code should be very similar, just with paths changed).

In [ ]:
import json
import numpy as np
import torch
import soundfile as sf

from pathlib import Path

from nemo.collections.tts.parts.utils.tts_dataset_utils import BetaBinomialInterpolator


def load_wav(audio_file, target_sr=None):
    with sf.SoundFile(audio_file, 'r') as f:
        samples = f.read(dtype='float32')
        sample_rate = f.samplerate
        if target_sr is not None and target_sr != sample_rate:
            samples = librosa.core.resample(samples, orig_sr=sample_rate, target_sr=target_sr)
    return samples.transpose()

# Get records from the training manifest
manifest_path = "./9017_manifest_train_dur_5_mins_local.json"
records = []
with open(manifest_path, "r") as f:
    for i, line in enumerate(f):
        records.append(json.loads(line))

beta_binomial_interpolator = BetaBinomialInterpolator()
spec_model.eval()

device = spec_model.device

save_dir = Path("./9017_manifest_train_dur_5_mins_local_mels")
save_dir.mkdir(exist_ok=True, parents=True)

# Generate a spectrograms (we need to use ground truth alignment for correct matching between audio and mels)
for i, r in enumerate(records):
    audio = load_wav(r["audio_filepath"])
    audio = torch.from_numpy(audio).unsqueeze(0).to(device)
    audio_len = torch.tensor(audio.shape[1], dtype=torch.long, device=device).unsqueeze(0)
    
    # Again, our finetuned FastPitch model doesn't use multiple speakers,
    # but we keep the code to support it here for reference
    if spec_model.fastpitch.speaker_emb is not None and "speaker" in r:
        speaker = torch.tensor([r['speaker']]).to(device)
    else:
        speaker = None
    
    with torch.no_grad():
        if "normalized_text" in r:
            text = spec_model.parse(r["normalized_text"], normalize=False)
        else:
            text = spec_model.parse(r['text'])
        
        text_len = torch.tensor(text.shape[-1], dtype=torch.long, device=device).unsqueeze(0)
    
        spect, spect_len = spec_model.preprocessor(input_signal=audio, length=audio_len)

        # Generate attention prior and spectrogram inputs for HiFi-GAN
        attn_prior = torch.from_numpy(
          beta_binomial_interpolator(spect_len.item(), text_len.item())
        ).unsqueeze(0).to(text.device)
            
        spectrogram = spec_model.forward(
          text=text, 
          input_lens=text_len, 
          spec=spect, 
          mel_lens=spect_len, 
          attn_prior=attn_prior,
          speaker=speaker,
        )[0]
        
        save_path = save_dir / f"mel_{i}.npy"
        np.save(save_path, spectrogram[0].to('cpu').numpy())
        r["mel_filepath"] = str(save_path)

hifigan_manifest_path = "hifigan_train_ft.json"
with open(hifigan_manifest_path, "w") as f:
    for r in records:
        f.write(json.dumps(r) + '\n')
# Please do the same for the validation json. Code is omitted.

We can then finetune hifigan similarly to fastpitch using NeMo's [hifigan_finetune.py](https://github.com/NVIDIA/NeMo/blob/main/examples/tts/hifigan_finetune.py) and [hifigan.yaml](https://github.com/NVIDIA/NeMo/blob/main/examples/tts/conf/hifigan/hifigan.yaml):

```bash
python examples/tts/hifigan_finetune.py \
--config-name=hifigan.yaml \
model.train_ds.dataloader_params.batch_size=32 \
model.max_steps=1000 \
model.optim.lr=0.00001 \
~model.optim.sched \
train_dataset=./hifigan_train_ft.json \
validation_datasets=./hifigan_val_ft.json \
exp_manager.exp_dir=hifigan_ft \
+init_from_pretrained_model=tts_en_hifigan \
trainer.check_val_every_n_epoch=10 \
model/train_ds=train_ds_finetune \
model/validation_ds=val_ds_finetune
```

Like when finetuning FastPitch, we lower the learning rate and get rid of the optimizer schedule for finetuning. You will need to create `<your_hifigan_val_manifest>` and the synthesized mels corresponding to it.

As mentioned, the above command is not runnable in Colab due to limited compute resources, but you are free to finetune HiFi-GAN on your own machines.

### Adding more data
We can add more data in two ways. They can be combined for the best effect:

* **Add more training data from the new speaker**

The entire notebook can be repeated from the top after a new JSON manifest is defined that includes the additional data. Modify your finetuning commands to point to the new manifest. Be sure to increase the number of steps as more data is added to both the FastPitch and HiFi-GAN finetuning.

We recommend **1000 steps per minute of audio for fastpitch and 500 steps per minute of audio for HiFi-GAN**.

* **Mix new speaker data with old speaker data**

We recommend finetuning FastPitch (but not HiFi-GAN) using both old speaker data (LJSpeech in this notebook) and the new speaker data. In this case, please modify the JSON manifests when finetuning FastPitch to include speaker information by adding a `speaker` field to each entry:

```
{"audio_filepath": "new_speaker.wav", "text": "sample", "duration": 2.6, "speaker": 1}
{"audio_filepath": "old_speaker.wav", "text": "LJSpeech sample", "duration": 2.6, "speaker": 0}
```

5 hours of data from the old speaker should be sufficient for training; it's up to you how much data from the old speaker to use in validation.

For the training manifest, since we likely have less data from the new speaker, we need to ensure that the model sees a similar amount of new data and old data. We can do this by repeating samples from the new speaker until we have an equivalent number of samples from the old and new speaker. For each sample from the old speaker, please add a sample from the new speaker in the .json.

As a toy example, if we use 4 samples of the old speaker and only 2 samples of the new speaker, we would want the order of samples in our manifest to look something like this:

```
old_speaker_sample_0
new_speaker_sample_0
old_speaker_sample_1
new_speaker_sample_1
old_speaker_sample_2
new_speaker_sample_0  # Start repeat of new speaker samples
old_speaker_sample_3
new_speaker_sample_1
```

Once the manifests are created, we can modify the FastPitch training command to point to the new training and validation JSON files.

We also need to update `model.n_speakers=1` to `model.n_speakers=2`, as well as update the `sup_data_types` specified in the config file to include `speaker_id` (`sup_data_types=[align_prior_matrix,pitch,speaker_id]`). Updating these two fields is very important--otherwise the model will not recognize that there is more than one speaker!

Ensure the pitch statistics correspond to the new speaker rather than the old speaker for best results.

**For HiFiGAN finetuning, the training should be done on the new speaker data.**